In [1]:
import tensorflow as tf 
from tensorflow.keras import datasets,layers,optimizers 

(xs,ys),_=datasets.mnist.load_data() 

/home/dske/anaconda3/envs/gluon/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dske/anaconda3/envs/gluon/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dske/anaconda3/envs/gluon/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/dske/anaconda3/envs/gluon/lib/python3.6/site-pack

In [2]:
print('datasets:',xs.shape,ys.shape) 

datasets: (60000, 28, 28) (60000,)


In [3]:
# 转化为tensor格式 
xs=tf.convert_to_tensor(xs,dtype=tf.float32)/255  
# 数据包装，方便进行batch
db=tf.data.Dataset.from_tensor_slices((xs,ys))

In [4]:
for step,(x,y) in enumerate(db):
    print(step,x.shape,y,y.shape) 
    break 

0 (28, 28) tf.Tensor(5, shape=(), dtype=uint8) ()


In [5]:
# 数据包装，方便进行batch,一次读取64
db=tf.data.Dataset.from_tensor_slices((xs,ys)).batch(64) 
# for step,(x,y) in enumerate(db):
#     print(step,x.shape,y,y.shape) 
#     break 

0 (64, 28, 28) tf.Tensor(
[5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9 4 0 9 1 1 2 4 3 2 7 3 8 6 9 0 5 6
 0 7 6 1 8 7 9 3 9 8 5 9 3 3 0 7 4 9 8 0 9 4 1 4 4 6 0], shape=(64,), dtype=uint8) (64,)


In [6]:
print(tf.__version__)

2.0.0-alpha0


In [7]:
model=tf.keras.Sequential([
    layers.Dense(512,activation='relu'),
    layers.Dense(256,activation='relu'),
    layers.Dense(10)
])
optimizer=optimizers.SGD(learning_rate=0.001) 


In [8]:
def train_epoch(epoch):
    for step,(x,y) in enumerate(db):
        with tf.GradientTape() as tape:
            x=tf.reshape(x,(-1,28*28))
            out=model(x) 
            print(out.shape) 
            print(y.shape) 
            loss=tf.reduce_sum(tf.square(out- tf.cast(y,tf.float32)))/x.shape[0]
        grads=tape.gradient(loss,model.trainable_variables) 
        optimizer.apply_gradients(zip(grads,model.trainable_variables))
        if step%100==0:
            print(epoch,step,loss.numpy()) 

In [9]:
def train():
    for epoch in range(30):
        train_epoch(epoch) 

In [10]:
train() 

(64, 10)
(64,)


InvalidArgumentError: Incompatible shapes: [64,10] vs. [64] [Op:Sub] name: sub/